In [1]:
import wandb

In [2]:
wandb.login()

True

In [3]:
import wandb
import os

In [4]:
os.getenv('WANDB_NOTEBOOK_NAME')

In [5]:
os.getenv('WANDB_NOTEBOOK_NAME')
os.environ['WANDB_NOTEBOOK_NAME'] = 'test'

In [6]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.visualization import (ZScaleInterval, ImageNormalize)
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from functions import plot_loss_curves
%load_ext autoreload
from func_job1 import *

In [7]:
cutout_dir = os.path.expandvars("$SCRATCH") + "/"
image_dir = "/home/anahoban/projects/rrg-kyi/astro/cfis/W3/"

In [8]:
#get tile ids
tile_list = open(image_dir + "tiles.list", "r")

# Only use tiles with all five channels
tile_list = open(image_dir + "tiles.list", "r")
tile_ids = []

for tile in tile_list:
    tile = tile[:-1] # Remove new line character
    channels = tile.split(" ")
    if len(channels) == 5: # Order is u,g,r,i,z
        tile_ids.append(channels[0][5:12]) # XXX.XXX id
tile_list.close()

In [9]:
hf= h5py.File(cutout_dir + "test.h5", "r")

In [10]:
hf = h5py.File(cutout_dir + "test.h5", "r")

In [11]:
BATCH_SIZE = 256 #128
CUTOUT_SIZE = 64
N_EPOCHS = 10

In [12]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.visualization import (ZScaleInterval, ImageNormalize)
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from functions import plot_loss_curves
%load_ext autoreload
from func_job2 import *

In [13]:
with open(cutout_dir + 'tiles_unbalanced.csv', newline='') as f:
    reader = csv.reader(f)
    data  = list(reader)
f.close()

tile_ids = data[0]

In [14]:
# 1. Start a W&B run
wandb.init(project='autoencoder tests', entity='mizar')

# 2. Save model inputs and hyperparameters
config = wandb.config
#config.learning_rate = 0.01

In [15]:
##TRAINING PREP##
BATCH_SIZE  = 256 
CUTOUT_SIZE = 64
N_EPOCHS    = 2 

# tiles for val and training
train_indices = range(1)
val_indices = [2]

#autosave
model_checkpoint_file = "../Models/job8.h5"
model_checkpoint_callback = ModelCheckpoint(model_checkpoint_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)


#training
bands = 2
autoencoder_cfis = create_autoencoder2((CUTOUT_SIZE, CUTOUT_SIZE, bands*2)) #last is the number of channels
autoencoder_cfis.compile(optimizer="adam", loss=MSE_with_uncertainty)

(autoencoder_cfis, history_cfis) = train_autoencoder(hf, tile_ids, autoencoder_cfis, train_indices,  val_indices, batch_size=BATCH_SIZE, cutout_size=CUTOUT_SIZE, n_epochs= N_EPOCHS, all_callbacks = [model_checkpoint_callback], bands="cfis")

hf.close()

In [16]:
def train_autoencoder(hf, tile_ids, model, train_indices, val_indices, n_epochs, batch_size, cutout_size, all_callbacks = None, bands="all"):
    n_cutouts_train = 0
    for i in train_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")   
        n_cutouts_train += len(img_group)
    
    n_cutouts_val = 0    
    for i in val_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")        
        n_cutouts_val += len(img_group)
    
    train_steps = n_cutouts_train // batch_size
    val_steps = n_cutouts_val // batch_size
    
    history = model.fit(get_cutouts(hf, tile_ids, train_indices, batch_size, cutout_size, bands), 
                        epochs=n_epochs, steps_per_epoch=train_steps, 
                        validation_data=get_cutouts(hf, tile_ids, val_indices, batch_size, cutout_size, bands), 
                        validation_steps=val_steps, callbacks= all_callbacks)
    
    # log metrics using wandb.log
    wandb.log({'epochs': len(history),
                   'loss': history['loss'][-1], 
                   'val_loss': history['val_loss'][-1]})
    return model, history

In [17]:
def train_auto(hf, tile_ids, model, train_indices, val_indices, n_epochs, batch_size, cutout_size, all_callbacks = None, bands="all"):
    n_cutouts_train = 0
    for i in train_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")   
        n_cutouts_train += len(img_group)
    
    n_cutouts_val = 0    
    for i in val_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")        
        n_cutouts_val += len(img_group)
    
    train_steps = n_cutouts_train // batch_size
    val_steps = n_cutouts_val // batch_size
    
    history = model.fit(get_cutouts(hf, tile_ids, train_indices, batch_size, cutout_size, bands), 
                        epochs=n_epochs, steps_per_epoch=train_steps, 
                        validation_data=get_cutouts(hf, tile_ids, val_indices, batch_size, cutout_size, bands), 
                        validation_steps=val_steps, callbacks= all_callbacks)
    
    # log metrics using wandb.log
    wandb.log({'epochs': len(history),
                   'loss': history['loss'][-1], 
                   'val_loss': history['val_loss'][-1]})
    return model, history

In [18]:
def train_auto(hf, tile_ids, model, train_indices, val_indices, n_epochs, batch_size, cutout_size, all_callbacks = None, bands="all"):
    n_cutouts_train = 0
    for i in train_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")   
        n_cutouts_train += len(img_group)
    
    n_cutouts_val = 0    
    for i in val_indices:
        img_group = hf.get(tile_ids[i] + "/IMAGES")        
        n_cutouts_val += len(img_group)
    
    train_steps = n_cutouts_train // batch_size
    val_steps = n_cutouts_val // batch_size
    
    history = model.fit(get_cutouts(hf, tile_ids, train_indices, batch_size, cutout_size, bands), 
                        epochs=n_epochs, steps_per_epoch=train_steps, 
                        validation_data=get_cutouts(hf, tile_ids, val_indices, batch_size, cutout_size, bands), 
                        validation_steps=val_steps, callbacks= all_callbacks)
    
    # log metrics using wandb.log
    wandb.log({'epochs': range(n_epochs),
                   'loss': history['loss'], 
                   'val_loss': history['val_loss']})
    return model, history

In [19]:
##TRAINING PREP##
BATCH_SIZE  = 256 
CUTOUT_SIZE = 64
N_EPOCHS    = 2 
configs = {
              "epochs": 2,
              "batch_size": 256,
              "cutout_size": 54,
              "architecture": "CNN",
              "dataset": "test"
           }
run = wandb.init(project='my-tf-integration', entity = 'mizar', config=configs)
config = wandb.config

# tiles for val and training
train_indices = range(1)
val_indices = [2]

#autosave
model_checkpoint_file = "../Models/job8.h5"
model_checkpoint_callback = ModelCheckpoint(model_checkpoint_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)


#training
bands = 2
autoencoder_cfis = create_autoencoder2((CUTOUT_SIZE, CUTOUT_SIZE, bands*2)) #last is the number of channels
autoencoder_cfis.compile(optimizer="adam", loss=MSE_with_uncertainty)

(autoencoder_cfis, history_cfis) = train_auto(hf, tile_ids, autoencoder_cfis, train_indices,  val_indices, batch_size=BATCH_SIZE, cutout_size=CUTOUT_SIZE, n_epochs= N_EPOCHS, all_callbacks = [model_checkpoint_callback], bands="cfis")

hf.close()

run.join()